In [ ]:
import torch as T
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

from NeuralNet import NeuralNet
from utils import *

from sklearn.datasets import fetch_covtype
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
class NeuralNet(nn.Module):
    def __init__(self, layers=[], loss_fn=None, device='cpu'):
        super(NeuralNet, self).__init__()

        # Prebacimo otpakovanu listu slojeva u nn.Sequential
        self.layers: nn.Sequential = nn.Sequential(*layers)

        # Definisimo funkciju gubitka za nas model
        self.loss_fn = loss_fn

        # Definisimo uredjaj na kojem treniramo model
        self.device = device
        self.to(device)

    def forward(self, x):
        # Proslijedimo ulaz kroz sve slojeve
        return self.layers(x)

    def train_model(self, x, y, optimizer, epochs=1000, early_stopping=True, early_stopping_epochs=3,  verbose=False):
        self.train()  # Treniranje modela

        epoch_loss_increase = 0
        previous_epoch_loss = 0
        minimum_loss = 0

        for epoch in range(epochs):
            if verbose:
                print(f'Epoch {epoch+1}/{epochs}')

            x = x.to(self.device)
            y = y.to(self.device)

            # Proslijedimo ulaz kroz sve slojeve
            y_pred = self.layers(x)

            # Izracunajmo gubitak i propagirajmo unazad
            loss = self.loss_fn(y_pred, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if verbose:
                print(f'Loss: {loss.item()}')

            # Provjerimo da li je greska porasla iznad kriterijuma
            if early_stopping:
                if loss.item() > previous_epoch_loss:
                    epoch_loss_increase += 1

                    if epoch_loss_increase == early_stopping_epochs:
                        print('Early stopping...')
                        break
                elif loss.item() < previous_epoch_loss:
                    minimum_loss = loss.item()
                    if verbose:
                        print(
                            f'---! NEW BEST EPOCH: !---\nCurrent: {minimum_loss}\nPrevious: {previous_epoch_loss}\n---! NEW BEST EPOCH: !---\n')
                    epoch_loss_increase = 0
                else:
                    epoch_loss_increase = 0

            # Sacuvajmo trenutnu vrijednost gubitka za sledeci pass
            previous_epoch_loss = loss.item()

        # Vratimo 'najbolji' rezultat minibatch-a
        return minimum_loss

    def test_model(self, x, y, verbose=True):
        self.eval()  # Testiranje modela

        total_loss = 0

        predictions = np.array([])

        with T.no_grad():
            for i in range(len(x)):
                if verbose:
                    print(f'Running test example [{i+1}/{len(x)}]')

                x = x.to(self.device)
                y = y.to(self.device)

                # Proslijedimo ulaz kroz sve slojeve
                y_pred = self.layers(x)
                predictions = np.append(predictions, y_pred)

                loss = self.loss_fn(y_pred, y)
                total_loss += loss.item()

                if verbose:
                    print(
                        f'[TEST_{i}] current loss: {loss.item()}, total_loss = {total_loss}')

        if verbose:
            print(f'Average loss: {total_loss/len(x)}')

        return predictions

    def predict(self, x):
        # Upitno ali ajde
        self.eval()  # Testiranje modela

        # Proslijedimo ulaz kroz sve slojeve
        return self.layers(x)


In [ ]:
X, y = fetch_covtype(return_X_y=True)
y = y.reshape((-1, 1))

X = StandardScaler().fit_transform(X)
y = StandardScaler().fit_transform(y)
y = y.reshape(-1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)
X_train, X_test, y_train, y_test = to_tensor(X_train), to_tensor(
    X_test), to_tensor(y_train, dtype=T.long), to_tensor(y_test, dtype=T.long)


In [ ]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

torch.Size([464809, 54]) torch.Size([464809]) torch.Size([116203, 54]) torch.Size([116203])


In [ ]:
train = TensorDataset(X_train, y_train)
test = TensorDataset(X_test, y_test)

batch_size = 64
train_loader = DataLoader(train, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test, batch_size=batch_size, shuffle=True)
device = T.device("cuda:0") if T.cuda.is_available() else T.device("cpu")

In [ ]:
loss_fn = nn.CrossEntropyLoss(reduction='mean')

nets = [
    NeuralNet(layers=[
      nn.Linear(54, 128),
      nn.ReLU(),
      nn.Dropout(0.1),

      nn.Linear(128, 128),
      nn.ReLU(),
      nn.Dropout(0.1),

      nn.Linear(128, 64),
      nn.GELU(),

      nn.Linear(64, 7),
      nn.LogSoftmax(dim=1),
  ], loss_fn=loss_fn, device=device),
    NeuralNet(layers=[
      nn.Linear(54, 128),
      nn.ReLU(),
      nn.Dropout(0.2),

      nn.Linear(128, 256),
      nn.ReLU(),
      nn.Dropout(0.1),

      nn.Linear(256, 7),
      nn.LogSoftmax(dim=1)
  ], loss_fn=loss_fn, device=device)]

lr = 1e-3
weight_decay = 1e-5

optim = T.optim.Adam(net.parameters(), lr=lr, weight_decay=weight_decay)

In [ ]:
num_epochs = 35
early_stopping_epochs = 3

batch = 0
for x, y in train_loader:
    print(f'Batch: {batch}')
    net.train_model(x, y, optim, num_epochs, early_stopping_epochs, verbose=True)
    batch += batch_size

In [ ]:
for x, y in test_loader:
    net.test_model(x, y, verbose=True)